<H1><b>Pertemuan 3 - Deep Learning Computation</b></H1>

<H3>Nama : Akmal Zuhdy Prasetya</H3>
<H3>NIM  : H071191035</H3>
<H5>Berdasarkan: <a>http://d2l.ai/chapter_deep-learning-computation/index.html<a><H5>

# **Import Library**

In [62]:
import torch
from torch import nn
from torch.nn import functional as F

# **1. Layers and Blocks**

**MLP**

In [63]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.0298, -0.3064, -0.1718,  0.1286,  0.0560, -0.3811,  0.0778,  0.1881,
          0.0587, -0.0960],
        [-0.0164, -0.2427, -0.1251,  0.1777, -0.0421, -0.2020,  0.0061,  0.0499,
         -0.0118, -0.0924]], grad_fn=<AddmmBackward0>)

## **1.1 A Custom Block**

In [64]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [65]:
net = MLP()
net(X)

tensor([[ 0.0721,  0.0100, -0.3043,  0.1977, -0.0716, -0.0512,  0.1893, -0.2582,
          0.0129,  0.2804],
        [ 0.2241,  0.1323, -0.1806,  0.0335, -0.1396,  0.0372,  0.0698, -0.1285,
          0.0034,  0.2503]], grad_fn=<AddmmBackward0>)

## **1.2 The Sequential Block**

In [66]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [67]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0737,  0.1467, -0.1244,  0.3718, -0.1908, -0.2584, -0.0944, -0.2627,
         -0.0133,  0.0866],
        [-0.0418,  0.0356, -0.1332,  0.4029, -0.3072, -0.2015, -0.0522, -0.1243,
          0.1106,  0.2236]], grad_fn=<AddmmBackward0>)

## **1.3 Executing Code in the Forward Propagation Function**

In [68]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
        
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /=2
        return X.sum()

In [69]:
net = FixedHiddenMLP()
net(X)

tensor(0.0016, grad_fn=<SumBackward0>)

In [70]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn. ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1690, grad_fn=<SumBackward0>)

# **2. Parameter Management**

## **2.1 Parameter Access**

In [71]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.1747],
        [-0.1797]], grad_fn=<AddmmBackward0>)

In [72]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.0712, -0.0271, -0.0533, -0.1187, -0.0261,  0.2001,  0.0247,  0.0779]])), ('bias', tensor([-0.0897]))])


In [73]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0897], requires_grad=True)
tensor([-0.0897])


In [74]:
net[2].weight.grad == None

True

In [75]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [76]:
net.state_dict()['2.bias'].data

tensor([-0.0897])

In [77]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.2344],
        [-0.2344]], grad_fn=<AddmmBackward0>)

In [78]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [79]:
rgnet[0][1][0].bias.data

tensor([-0.1841, -0.4663,  0.1562, -0.0297,  0.4831, -0.0442, -0.2568, -0.2047])

## **2.2 Parameter Initialization**

### **2.2.1 Built-in Initialization**

In [80]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0165,  0.0054, -0.0027,  0.0200]), tensor(0.))

In [81]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [82]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
        
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.1352, -0.6523, -0.1155, -0.4591])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### **2.2.2 Custom Initialization**

In [83]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >=5
        
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 6.9860,  0.0000,  0.0000, -7.2898],
        [ 9.6665,  0.0000,  8.1232,  7.6009]], grad_fn=<SliceBackward0>)

In [84]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  1.0000, -6.2898])

### **2.2.3 Tied Parameters**

In [85]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)

print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


# **4. Custom Layers**

## **4.1 Layers without Parameters**

In [86]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, X):
        return X - X.mean()

In [87]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [88]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [89]:
Y =net(torch.rand(4, 8))
Y.mean()

tensor(3.7253e-09, grad_fn=<MeanBackward0>)

## **4.2 Layers with Parameters**

In [90]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
        
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [91]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.7676, -0.0545, -0.5944],
        [-0.9176,  0.0344,  1.6060],
        [ 0.5840,  0.5776,  0.7196],
        [-0.1227,  0.2799, -1.1174],
        [ 1.6724, -2.6224,  0.9776]], requires_grad=True)

In [92]:
linear(torch.rand(2, 5))

tensor([[3.1517, 0.0000, 1.2425],
        [2.6486, 0.0000, 2.1800]])

In [93]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])

# **5. File I/O**

## **5.1 Loading and Saving Tensors**

In [94]:
x = torch.arange(4)
torch.save(x, 'x-file')

In [95]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [96]:
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [97]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## **5.2 Loading and Saving Model Parameters**

In [98]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))
    
net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [99]:
torch.save(net.state_dict(), 'mlp.params')

In [100]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [101]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

# **6. GPUs**

## **6.1 Computing Devices**

In [102]:
torch.device('cpu'), torch.device('cuda')

(device(type='cpu'), device(type='cuda'))

In [103]:
torch.cuda.device_count()

1

In [104]:
def try_gpu(i=0):
    """Return gpu(i) if exists, otherwise return cpu()."""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

## **6.2 Tensors and GPUs**

In [105]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [106]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [107]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.7180, 0.2979, 0.2464],
        [0.2757, 0.1571, 0.9068]])

In [108]:
Z = X # Z = X.cuda(1)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [109]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [110]:
net(X)

tensor([[0.6824],
        [0.6824]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [111]:
net[0].weight.data.device

device(type='cuda', index=0)